In [188]:
# Basics
import pandas as pd
import numpy as np
import time
import os
from os import listdir
from os.path import isfile, join, basename
from tqdm import tqdm

import math
import sys
import datetime

# Helper functions
from lag_features import *
from other_functions import *

# Preprocessing
from sklearn.preprocessing import LabelEncoder

# Models
#from sklearn.ensemble import GradientBoostingRegressor as gbr
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Metrics
from sklearn.metrics import mean_squared_error


OSError: dlopen(/Users/carlosperezricardo/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/carlosperezricardo/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [96]:
DIR = '/Users/carlosperezricardo/Desktop/TFM'

#[10397, 13930, 11298, 11292, 12892]
AIRPORT = 10397
CARRIER = 19393
years_to_load = ['2016','2017','2018','2019']

In [97]:
folder = os.path.join(DIR, 'datasets')
files = [f for f in listdir(folder) if isfile(join(folder, f))]
files = sorted(files)

In [98]:
df = pd.DataFrame()
for file in files:
    loading = []
    if '.zip' in file:
        for year in years_to_load:
            loading.append(year in file)
        output = any(loading)
        if output:
            add_df = pd.read_csv(os.path.join(DIR,'datasets',file), parse_dates=['FL_DATE'])
            add_df = add_df[(add_df.DEST_AIRPORT_ID == AIRPORT)]
            df = pd.concat([df, add_df], axis=0)
df.reset_index(drop=True, inplace=True)

/Users/carlosperezricardo/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [99]:
df.head()

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,...,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 29
0,2016-01-01,AA,19805,AA,N3KVAA,1178,11298,1129804,30194,DFW,...,24.0,-3.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,2016-01-01,AA,19805,AA,N3AUAA,1427,11298,1129804,30194,DFW,...,1205.0,-10.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,2016-01-01,AA,19805,AA,N479AA,1442,11298,1129804,30194,DFW,...,1439.0,42.0,0.0,NaN,0.0,1.0,0.0,0.0,41.0,NaN
3,2016-01-01,AS,19930,AS,N477AS,746,14747,1474703,30559,SEA,...,1736.0,-2.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,2016-01-01,AS,19930,AS,N462AS,750,14747,1474703,30559,SEA,...,2354.0,-4.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [100]:
df['DEST'].value_counts()

ATL    1500685
Name: DEST, dtype: int64

In [101]:
ini_date = datetime.datetime(2016, 1, 1)

train_ini_date = datetime.datetime(2018, 1, 1)
train_fin_date = datetime.datetime(2019, 9, 1)

test_ini_date = datetime.datetime(2019, 9, 1)
test_fin_date = datetime.datetime(2019, 9, 30)

In [102]:
df = df[(df.FL_DATE > ini_date) & (df.FL_DATE <= test_fin_date)]

In [103]:
df.shape

(1402736, 30)

In [104]:
# Feature Generation 
df = date_features(df, 'FL_DATE')

In [105]:
# Add flight distance in km
def flight_distance(lat1, lon1, lat2, lon2):
    R = 6371
    phi1 = lat1 * math.pi/180
    phi2 = lat2 * math.pi/180
    delta_phi = (lat2-lat1) * math.pi/180
    delta_lambda = (lon2-lon1) * math.pi/180
    
    a = np.sin(delta_phi/2) * np.sin(delta_lambda/2) + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda/2) * np.sin(delta_lambda/2);
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    return R * c # in kilometres

In [106]:
airports_df = pd.read_csv(os.path.join('/Users/carlosperezricardo/Desktop/TFM/airlines_airports', 'L_AIRPORT_ID.csv'))
airports_df = airports_df[['Code','Description','City_State','City','State_Country','lat','lon']]

In [107]:
df = pd.merge(df, airports_df[['Code','lat','lon']], left_on='ORIGIN_AIRPORT_ID', right_on='Code')
df.rename({'lat':'lat_ORIGIN','lon':'lon_ORIGIN'}, axis=1, inplace=True)

df = pd.merge(df, airports_df[['Code','lat','lon']], left_on='DEST_AIRPORT_ID', right_on='Code')
df.rename({'lat':'lat_DEST','lon':'lon_DEST'}, axis=1, inplace=True)

In [108]:
df['flight_distance'] = flight_distance(df['lat_ORIGIN'], df['lon_ORIGIN'], df['lat_DEST'], df['lon_DEST'])

/Users/carlosperezricardo/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [109]:
# Add weather
WEATHER_DIR = '/Users/carlosperezricardo/Desktop/TFM/weather'
weather_df = pd.read_csv(os.path.join(WEATHER_DIR, 'weather.csv'))

In [110]:
iata_code = list(df['DEST'].unique())[0]
iata_code

'ATL'

In [111]:
airports_df = pd.read_csv(os.path.join('/Users/carlosperezricardo/Desktop/TFM/airlines_airports', 'airports.csv'))

airports_code = pd.read_excel(os.path.join('/Users/carlosperezricardo/Desktop/TFM/airlines_airports', 'US_airports.xlsx'), sheet_name='Airports')
airports_code.columns = airports_code.iloc[0]
airports_code = airports_code.iloc[1:]
    
states = pd.read_excel(os.path.join('/Users/carlosperezricardo/Desktop/TFM/airlines_airports', 'US_airports.xlsx'), sheet_name='States')
states.columns = ['state_name','state_acronym']

airports = pd.merge(airports_code, states, left_on='City', right_on='state_name', how='left')

airports[['state_name','state_acronym']] = airports[['state_name','state_acronym']].ffill(axis = 0)
airports = airports[~airports['FAA'].isnull()]

In [112]:
icao_code = list(airports[airports.IATA == iata_code]['ICAO'])[0]
icao_code

'KATL'

In [113]:
weather_df = weather_df[weather_df.ICAO == icao_code]
weather_df['Day'] = pd.to_datetime(weather_df['Day'])
weather_df.head()

,Unnamed: 0,Day,Temperature_Max,Temperature_Avg,Temperature_Min,Dew Point_Max,Dew Point_Avg,Dew Point_Min,Humidity_Max,Humidity_Avg,Humidity_Min,Wind Speed_Max,Wind Speed_Avg,Wind Speed_Min,Pressure_Max,Pressure_Avg,Pressure_Min,Precipitation,ICAO
85792,85792,2019-01-01,59,53.8,49,53,49.0,44,93,84.3,69,9.0,5.4,0.0,29.1,29.1,28.9,0.00,KATL
85793,85793,2019-01-02,55,53.1,50,53,50.6,49,96,91.3,86,14.0,4.6,0.0,29.0,28.9,28.9,0.60,KATL
85794,85794,2019-01-03,66,55.8,45,58,50.7,37,97,83.8,58,25.0,10.9,3.0,28.9,28.7,28.6,1.00,KATL
85795,85795,2019-01-04,59,49.0,44,38,36.4,34,79,63.3,41,21.0,14.2,7.0,29.0,28.9,28.7,0.56,KATL
85796,85796,2019-01-05,69,54.5,43,41,38.2,36,80,56.7,33,10.0,6.6,0.0,29.1,29.1,29.0,0.00,KATL


In [114]:
df = pd.merge(df, weather_df, left_on='FL_DATE', right_on='Day', how='left')

In [115]:
df.head()

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,...,Humidity_Avg,Humidity_Min,Wind Speed_Max,Wind Speed_Avg,Wind Speed_Min,Pressure_Max,Pressure_Avg,Pressure_Min,Precipitation,ICAO
0,2016-01-02,AA,19805,AA,N471AA,232,11298,1129804,30194,DFW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-02,AA,19805,AA,N3JXAA,1487,11298,1129804,30194,DFW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-02,AA,19805,AA,N472AA,1641,11298,1129804,30194,DFW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-02,AA,19805,AA,N3CHAA,2408,11298,1129804,30194,DFW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-02,DL,19790,DL,N952DN,30,11298,1129804,30194,DFW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
df = df[(df.CANCELLED != 1) & (df.DIVERTED != 1)]

In [117]:
df['DEP_TIME'] = df['DEP_TIME'].astype(int)/100
df['ARR_TIME'] = df['ARR_TIME'].astype(int)/100

df['airport_DEP_n_flights_per_hour'] = df.groupby(['FL_DATE','DEP_TIME'])['ARR_TIME'].transform( lambda x: x.count())
df['airport_ARR_n_flights_per_hour'] = df.groupby(['FL_DATE','ARR_TIME'])['DEP_TIME'].transform( lambda x: x.count())

df['airport_n_flights_per_hour'] = df['airport_DEP_n_flights_per_hour'] + df['airport_ARR_n_flights_per_hour'] 

df['airport_DEP_n_flights_cumsum'] = df.groupby(['FL_DATE'])['airport_DEP_n_flights_per_hour'].transform( lambda x: x.cumsum())
df['airport_ARR_n_flights_cumsum'] = df.groupby(['FL_DATE'])['airport_ARR_n_flights_per_hour'].transform( lambda x: x.cumsum())
df['airport_n_flights_cumsum'] = df.groupby(['FL_DATE'])['airport_n_flights_per_hour'].transform( lambda x: x.cumsum())

In [118]:
df = df[df.OP_CARRIER_AIRLINE_ID == CARRIER]

In [119]:
df.shape

(150937, 68)

In [120]:
df.head()

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,...,Pressure_Avg,Pressure_Min,Precipitation,ICAO,airport_DEP_n_flights_per_hour,airport_ARR_n_flights_per_hour,airport_n_flights_per_hour,airport_DEP_n_flights_cumsum,airport_ARR_n_flights_cumsum,airport_n_flights_cumsum
25420,2016-01-02,WN,19393,WN,N240WN,5270,12892,1289203,32575,LAX,...,NaN,NaN,NaN,NaN,3.0,2.0,5.0,61.0,55.0,116.0
25421,2016-01-02,WN,19393,WN,N446WN,5863,12892,1289203,32575,LAX,...,NaN,NaN,NaN,NaN,12.0,2.0,14.0,73.0,57.0,130.0
25422,2016-01-02,WN,19393,WN,N402WN,5981,12892,1289203,32575,LAX,...,NaN,NaN,NaN,NaN,1.0,2.0,3.0,74.0,59.0,133.0
25425,2016-01-03,WN,19393,WN,N950WN,4097,12892,1289203,32575,LAX,...,NaN,NaN,NaN,NaN,3.0,3.0,6.0,42.0,50.0,92.0
25426,2016-01-03,WN,19393,WN,N283WN,4557,12892,1289203,32575,LAX,...,NaN,NaN,NaN,NaN,2.0,2.0,4.0,44.0,52.0,96.0


In [121]:
df.columns

Index(['FL_DATE', 'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER',
       'TAIL_NUM', 'OP_CARRIER_FL_NUM', 'ORIGIN_AIRPORT_ID',
       'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID', 'ORIGIN',
       'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'DEST_AIRPORT_ID',
       'DEST_AIRPORT_SEQ_ID', 'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME',
       'DEST_STATE_ABR', 'DEP_TIME', 'DEP_DELAY', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'LATE_AIRCRAFT_DELAY', 'Unnamed: 29',
       'month', 'day', 'year', 'FL_DATE_quarter', 'weekday', 'year_month',
       'Code_x', 'lat_ORIGIN', 'lon_ORIGIN', 'Code_y', 'lat_DEST', 'lon_DEST',
       'flight_distance', 'Unnamed: 0', 'Day', 'Temperature_Max',
       'Temperature_Avg', 'Temperature_Min', 'Dew Point_Max', 'Dew Point_Avg',
       'Dew Point_Min', 'Humidity_Max', 'Humidity_Avg', 'Humidity_Min',
       'Wind Speed_Max', 'Wind Speed_Avg', 'Wind Speed_Min

In [122]:
df_ = df.copy(deep=True)

In [123]:
#df_['ARR_DELAY_'] = df_['ARR_DELAY'] > 15
#df_['DEP_DELAY_'] = df_['DEP_DELAY'] > 15

In [124]:
s1 = 7
s2 = 14

In [125]:
# Lag features by: TAIL_NUM
calculations = {}
calculations['calc1'] = {'gb_list':['TAIL_NUM','FL_DATE'],'target':'DEP_DELAY','shifts':[s1], 'windows':[3,5,10], 'funs':['mean']}

df_ = apply_calc(df_, calculations)

Generating DEP_DELAY_TAIL_NUM_FL_DATE_s7_r3_mean
Generating DEP_DELAY_TAIL_NUM_FL_DATE_s7_r5_mean
Generating DEP_DELAY_TAIL_NUM_FL_DATE_s7_r10_mean


In [126]:
# Lag features by: DEST_AIRPORT_ID
calculations = {}
calculations['calc2'] = {'gb_list':['FL_DATE'],'target':'DEP_DELAY','shifts':[s1,s2], 'windows':[1,5,10], 'funs':['mean','std']}
calculations['calc3'] = {'gb_list':['FL_DATE'],'target':'DEP_DELAY','shifts':[s1,s2], 'windows':[1,15,30], 'funs':['median','std']}

df_ = apply_calc(df_, calculations)

Generating DEP_DELAY_FL_DATE_s7_r1_mean
Generating DEP_DELAY_FL_DATE_s14_r1_mean
Generating DEP_DELAY_FL_DATE_s7_r5_mean
Generating DEP_DELAY_FL_DATE_s14_r5_mean
Generating DEP_DELAY_FL_DATE_s7_r10_mean
Generating DEP_DELAY_FL_DATE_s14_r10_mean
Generating DEP_DELAY_FL_DATE_s7_r1_std
Generating DEP_DELAY_FL_DATE_s14_r1_std
Generating DEP_DELAY_FL_DATE_s7_r5_std
Generating DEP_DELAY_FL_DATE_s14_r5_std
Generating DEP_DELAY_FL_DATE_s7_r10_std
Generating DEP_DELAY_FL_DATE_s14_r10_std
Generating DEP_DELAY_FL_DATE_s7_r1_median
Generating DEP_DELAY_FL_DATE_s14_r1_median
Generating DEP_DELAY_FL_DATE_s7_r15_median
Generating DEP_DELAY_FL_DATE_s14_r15_median
Generating DEP_DELAY_FL_DATE_s7_r30_median
Generating DEP_DELAY_FL_DATE_s14_r30_median
Generating DEP_DELAY_FL_DATE_s7_r1_std
Generating DEP_DELAY_FL_DATE_s14_r1_std
Generating DEP_DELAY_FL_DATE_s7_r15_std
Generating DEP_DELAY_FL_DATE_s14_r15_std
Generating DEP_DELAY_FL_DATE_s7_r30_std
Generating DEP_DELAY_FL_DATE_s14_r30_std


In [127]:
# Lag features by: OP_CARRIER_AIRLINE_ID
calculations = {}
calculations['calc4'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'DEP_DELAY','shifts':[s1,s2], 'windows':[1,5,10], 'funs':['mean']}
calculations['calc5'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'DEP_DELAY','shifts':[s2], 'windows':[10], 'funs':['median','std']}

df_ = apply_calc(df_, calculations)

Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_r1_mean
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s14_r1_mean
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_r5_mean
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s14_r5_mean
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_r10_mean
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s14_r10_mean
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s14_r10_median
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s14_r10_std


In [128]:
calculations = {}
calculations['calc4'] = {'gb_list':['ORIGIN','FL_DATE'],'target':'DEP_DELAY','shifts':[s1,s2], 'windows':[1,5,10], 'funs':['mean']}
calculations['calc5'] = {'gb_list':['ORIGIN','FL_DATE'],'target':'DEP_DELAY','shifts':[s1,s2], 'windows':[10], 'funs':['median','std']}

df_ = apply_calc(df_, calculations)

Generating DEP_DELAY_ORIGIN_FL_DATE_s7_r1_mean
Generating DEP_DELAY_ORIGIN_FL_DATE_s14_r1_mean
Generating DEP_DELAY_ORIGIN_FL_DATE_s7_r5_mean
Generating DEP_DELAY_ORIGIN_FL_DATE_s14_r5_mean
Generating DEP_DELAY_ORIGIN_FL_DATE_s7_r10_mean
Generating DEP_DELAY_ORIGIN_FL_DATE_s14_r10_mean
Generating DEP_DELAY_ORIGIN_FL_DATE_s7_r10_median
Generating DEP_DELAY_ORIGIN_FL_DATE_s14_r10_median
Generating DEP_DELAY_ORIGIN_FL_DATE_s7_r10_std
Generating DEP_DELAY_ORIGIN_FL_DATE_s14_r10_std


In [129]:
# Convert times 
# [ARR_TIME, DEP_TIME]

df_['DEP_TIME'] = df_['DEP_TIME']/100
df_['ARR_TIME'] = df_['ARR_TIME']/100
df_['DEP_TIME'] = df_['DEP_TIME'].astype(int)
df_['ARR_TIME'] = df_['ARR_TIME'].astype(int)

In [130]:
df_['DEP_n_flights_per_hour'] = df_.groupby(['FL_DATE','DEP_TIME'])['ARR_TIME'].transform( lambda x: x.count())
df_['ARR_n_flights_per_hour'] = df_.groupby(['FL_DATE','ARR_TIME'])['DEP_TIME'].transform( lambda x: x.count())

df_['n_flights_per_hour'] = df_['DEP_n_flights_per_hour'] + df_['ARR_n_flights_per_hour'] 

df_['DEP_n_flights_cumsum'] = df_.groupby(['FL_DATE'])['DEP_n_flights_per_hour'].transform( lambda x: x.cumsum())
df_['ARR_n_flights_cumsum'] = df_.groupby(['FL_DATE'])['ARR_n_flights_per_hour'].transform( lambda x: x.cumsum())
df_['n_flights_cumsum'] = df_.groupby(['FL_DATE'])['n_flights_per_hour'].transform( lambda x: x.cumsum())

In [131]:
# Not Lagged
calculations = {}
calculations['calc6'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'DEP_DELAY','shifts':[s1], 'funs':['mean','std']}
calculations['calc7'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'DEP_DELAY','shifts':[s1], 'funs':['mean','std']}

df_ = apply_calc(df_, calculations)

Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_mean
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_std
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_mean
Generating DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_std


In [132]:
# Type of delay
calculations = {}
calculations['calc1'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'CARRIER_DELAY','shifts':[s1], 'funs':['mean']}
calculations['calc2'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'LATE_AIRCRAFT_DELAY','shifts':[s1], 'funs':['median']}
calculations['calc3'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'NAS_DELAY','shifts':[s1], 'funs':['mean']}
calculations['calc4'] = {'gb_list':['FL_DATE'],'target':'WEATHER_DELAY','shifts':[s1], 'funs':['mean']}
calculations['calc5'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'LATE_AIRCRAFT_DELAY','shifts':[s1], 'funs':['mean']}

calculations['calc6'] = {'gb_list':['ORIGIN_AIRPORT_ID'],'target':'WEATHER_DELAY','shifts':[s1], 'funs':['mean']}
calculations['calc7'] = {'gb_list':['ORIGIN_AIRPORT_ID'],'target':'NAS_DELAY','shifts':[s1], 'funs':['mean']}

df_ = apply_calc(df_, calculations)

Generating CARRIER_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_mean
Generating LATE_AIRCRAFT_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_median
Generating NAS_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_mean
Generating WEATHER_DELAY_FL_DATE_s7_mean
Generating LATE_AIRCRAFT_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_mean
Generating WEATHER_DELAY_ORIGIN_AIRPORT_ID_s7_mean
Generating NAS_DELAY_ORIGIN_AIRPORT_ID_s7_mean


In [133]:
calculations = {}
calculations['calc'] = {'gb_list':['FL_DATE'],'target':'DEP_DELAY','shifts':[s1, s2], 'windows':[1,7,14], 'funs':['mean','std']}

df_ = apply_calc(df_, calculations)

Generating DEP_DELAY_FL_DATE_s7_r1_mean
Generating DEP_DELAY_FL_DATE_s14_r1_mean
Generating DEP_DELAY_FL_DATE_s7_r7_mean
Generating DEP_DELAY_FL_DATE_s14_r7_mean
Generating DEP_DELAY_FL_DATE_s7_r14_mean
Generating DEP_DELAY_FL_DATE_s14_r14_mean
Generating DEP_DELAY_FL_DATE_s7_r1_std
Generating DEP_DELAY_FL_DATE_s14_r1_std
Generating DEP_DELAY_FL_DATE_s7_r7_std
Generating DEP_DELAY_FL_DATE_s14_r7_std
Generating DEP_DELAY_FL_DATE_s7_r14_std
Generating DEP_DELAY_FL_DATE_s14_r14_std


In [134]:
drop_cols = ['CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'LATE_AIRCRAFT_DELAY', 'Unnamed: 29']
for col in drop_cols:
    del df_[col]

In [135]:
keep = list(df_.select_dtypes(include=np.number).columns)

for col in df_.select_dtypes(exclude=np.number).columns:
    print(col, len(df_[col].unique()))
    if len(df_[col].unique()) < 100:
        keep.append(col)
        le = LabelEncoder()
        df_[col] = le.fit_transform(df_[col])
        df_[col] = df_[col].astype('category')

FL_DATE 1338
OP_UNIQUE_CARRIER 1
OP_CARRIER 1
TAIL_NUM 873
ORIGIN 40
ORIGIN_CITY_NAME 39
ORIGIN_STATE_ABR 24
DEST 1
DEST_CITY_NAME 1
DEST_STATE_ABR 1
year_month 44
Day 274
ICAO 2


In [136]:
drop_cols = ['DEST_CITY_NAME','ORIGIN_CITY_NAME', 'Unnamed: 0','year_month']

for col in drop_cols:
    try:
        keep.remove(col)
    except:
        pass


In [137]:
df_.fillna(0, inplace=True)

In [138]:
df_.head()

,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,...,WEATHER_DELAY_ORIGIN_AIRPORT_ID_s7_mean,NAS_DELAY_ORIGIN_AIRPORT_ID_s7_mean,DEP_DELAY_FL_DATE_s7_r7_mean,DEP_DELAY_FL_DATE_s14_r7_mean,DEP_DELAY_FL_DATE_s7_r14_mean,DEP_DELAY_FL_DATE_s14_r14_mean,DEP_DELAY_FL_DATE_s7_r7_std,DEP_DELAY_FL_DATE_s14_r7_std,DEP_DELAY_FL_DATE_s7_r14_std,DEP_DELAY_FL_DATE_s14_r14_std
25420,2016-01-02,0,19393,0,N240WN,5270,12892,1289203,32575,18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25421,2016-01-02,0,19393,0,N446WN,5863,12892,1289203,32575,18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25422,2016-01-02,0,19393,0,N402WN,5981,12892,1289203,32575,18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25425,2016-01-03,0,19393,0,N950WN,4097,12892,1289203,32575,18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25426,2016-01-03,0,19393,0,N283WN,4557,12892,1289203,32575,18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [139]:
train_df = df_.loc[(df_.FL_DATE > datetime.datetime(2018, 1, 1)) & (df_.FL_DATE < datetime.datetime(2019, 9, 1)), keep]
test_df = df_.loc[(df_.FL_DATE >= datetime.datetime(2019, 9, 1)) & (df_.FL_DATE <= datetime.datetime(2019, 9, 30)), keep]

In [140]:
print(train_df.shape)
print(test_df.shape)

(67793, 119)
(3061, 119)


In [141]:
TARGET = 'DEP_DELAY'
drop = ['DEP_DELAY','ARR_DELAY']

In [142]:
features = [ col for col in train_df.columns if col != TARGET and col not in drop ]

X_train = train_df[features]
y_train = train_df[TARGET]

X_test = test_df[features]
y_test = test_df[TARGET]

In [143]:
reg = gbr(loss='absolute_error', n_estimators=200)
reg.fit(X_train, y_train)
x = model_metrics(X_train, y_train, X_test, y_test, reg)

 --- TRAIN --- 
     - RMSE:  982.124263886868
     - MAE:  12.712866220813998
     - R2:  -0.06048236952542685
 --- TEST --- 
     - RMSE:  361.79310835210043
     - MAE:  7.731304547931059
     - R2:  -0.03354853394189017


In [144]:
reg = gbr(loss='squared_error', n_estimators=200)
reg.fit(X_train, y_train)
x = model_metrics(X_train, y_train, X_test, y_test, reg)

 --- TRAIN --- 
     - RMSE:  816.723273626254
     - MAE:  15.254494805451186
     - R2:  0.11811502443289312
 --- TEST --- 
     - RMSE:  371.3157826802074
     - MAE:  11.468364736562837
     - R2:  -0.06075233043168571


In [145]:
model = reg

In [146]:
model = gbr(random_state=42)

gridParams = {
    'n_estimators': [200, 250, 300],
    'max_depth': [10, 20, 30],
    'num_leaves':[20, 30, 40]#,
    #'loss':['squared_error','absolute_error']
}

grid = GridSearchCV(model, gridParams, verbose=1)
# Run the grid
grid.fit(X_train, y_train)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)

model = grid.best_estimator_

Fitting 5 folds for each of 27 candidates, totalling 135 fits


ValueError: Invalid parameter num_leaves for estimator GradientBoostingRegressor(max_depth=10, n_estimators=200, random_state=42). Check the list of available parameters with `estimator.get_params().keys()`.

In [148]:
model = reg

In [149]:
pd.DataFrame({'columns': X_train.columns,'feature_importance':model.feature_importances_}).sort_values('feature_importance',ascending=False).head(25)

,columns,feature_importance
93,DEP_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_mean,0.205868
40,airport_n_flights_per_hour,0.135009
97,NAS_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_mean,0.056280
1,OP_CARRIER_FL_NUM,0.035894
101,NAS_DELAY_ORIGIN_AIRPORT_ID_s7_mean,0.034148
100,WEATHER_DELAY_ORIGIN_AIRPORT_ID_s7_mean,0.030351
105,DEP_DELAY_FL_DATE_s14_r14_mean,0.027143
63,DEP_DELAY_FL_DATE_s7_r30_median,0.025587
38,airport_DEP_n_flights_per_hour,0.022758
10,month,0.019871


In [150]:
x = model_metrics(X_train, y_train, X_test, y_test, model)

 --- TRAIN --- 
     - RMSE:  816.723273626254
     - MAE:  15.254494805451186
     - R2:  0.11811502443289312
 --- TEST --- 
     - RMSE:  371.3157826802074
     - MAE:  11.468364736562837
     - R2:  -0.06075233043168571


In [151]:
train_df_ = train_df.copy(deep=True)
train_df_['pred'] = reg.predict(X_train)
train_df_[['DEP_DELAY','pred']].sort_values('DEP_DELAY', ascending=False).head(30)

,DEP_DELAY,pred
167736,500.0,10.801282
43693,486.0,10.192161
39143,479.0,12.124447
429083,470.0,2.695511
1019599,462.0,2.147632
298109,453.0,25.924867
554615,433.0,8.683569
1075239,431.0,14.060411
499468,416.0,35.100722
1032656,412.0,56.651794


In [152]:
train_df_[['DEP_DELAY','pred']].sort_values('DEP_DELAY', ascending=True).head(30)

,DEP_DELAY,pred
461599,-20.0,4.716314
762510,-18.0,6.322034
428413,-15.0,5.039362
629179,-15.0,10.595571
924235,-14.0,10.588550
463026,-13.0,2.464375
462040,-13.0,3.690988
924060,-13.0,2.943405
697454,-13.0,7.202932
760863,-13.0,-3.615882


In [153]:
# ARR_DELAY

In [154]:
df_ = df.copy(deep=True)

In [155]:
df_ = df_[(df_.CANCELLED != 1) & (df_.DIVERTED != 1)]

In [156]:
s1 = 7
s2 = 14
s3 = 365

In [157]:
# Lag features by: TAIL_NUM
calculations = {}
calculations['calc1'] = {'gb_list':['TAIL_NUM','FL_DATE'],'target':'ARR_DELAY','shifts':[s1], 'windows':[3,5,10], 'funs':['mean']}

df_ = apply_calc(df_, calculations)

Generating ARR_DELAY_TAIL_NUM_FL_DATE_s7_r3_mean
Generating ARR_DELAY_TAIL_NUM_FL_DATE_s7_r5_mean
Generating ARR_DELAY_TAIL_NUM_FL_DATE_s7_r10_mean


In [158]:
# Lag features by: DEST_AIRPORT_ID
calculations = {}
calculations['calc2'] = {'gb_list':['FL_DATE'],'target':'ARR_DELAY','shifts':[s1,s2], 'windows':[1,5,10], 'funs':['mean','std']}
calculations['calc3'] = {'gb_list':['FL_DATE'],'target':'ARR_DELAY','shifts':[s1,s2], 'windows':[1,15,30], 'funs':['median','std']}

df_ = apply_calc(df_, calculations)

Generating ARR_DELAY_FL_DATE_s7_r1_mean
Generating ARR_DELAY_FL_DATE_s14_r1_mean
Generating ARR_DELAY_FL_DATE_s7_r5_mean
Generating ARR_DELAY_FL_DATE_s14_r5_mean
Generating ARR_DELAY_FL_DATE_s7_r10_mean
Generating ARR_DELAY_FL_DATE_s14_r10_mean
Generating ARR_DELAY_FL_DATE_s7_r1_std
Generating ARR_DELAY_FL_DATE_s14_r1_std
Generating ARR_DELAY_FL_DATE_s7_r5_std
Generating ARR_DELAY_FL_DATE_s14_r5_std
Generating ARR_DELAY_FL_DATE_s7_r10_std
Generating ARR_DELAY_FL_DATE_s14_r10_std
Generating ARR_DELAY_FL_DATE_s7_r1_median
Generating ARR_DELAY_FL_DATE_s14_r1_median
Generating ARR_DELAY_FL_DATE_s7_r15_median
Generating ARR_DELAY_FL_DATE_s14_r15_median
Generating ARR_DELAY_FL_DATE_s7_r30_median
Generating ARR_DELAY_FL_DATE_s14_r30_median
Generating ARR_DELAY_FL_DATE_s7_r1_std
Generating ARR_DELAY_FL_DATE_s14_r1_std
Generating ARR_DELAY_FL_DATE_s7_r15_std
Generating ARR_DELAY_FL_DATE_s14_r15_std
Generating ARR_DELAY_FL_DATE_s7_r30_std
Generating ARR_DELAY_FL_DATE_s14_r30_std


In [159]:
# Lag features by: ORIGIN
calculations = {}
calculations['calc4'] = {'gb_list':['ORIGIN'],'target':'ARR_DELAY','shifts':[s1,s2], 'windows':[1,5,10], 'funs':['mean','std']}
calculations['calc5'] = {'gb_list':['ORIGIN'],'target':'ARR_DELAY','shifts':[s1,s2], 'windows':[1,15,30], 'funs':['median','std']}

df_ = apply_calc(df_, calculations)

Generating ARR_DELAY_ORIGIN_s7_r1_mean
Generating ARR_DELAY_ORIGIN_s14_r1_mean
Generating ARR_DELAY_ORIGIN_s7_r5_mean
Generating ARR_DELAY_ORIGIN_s14_r5_mean
Generating ARR_DELAY_ORIGIN_s7_r10_mean
Generating ARR_DELAY_ORIGIN_s14_r10_mean
Generating ARR_DELAY_ORIGIN_s7_r1_std
Generating ARR_DELAY_ORIGIN_s14_r1_std
Generating ARR_DELAY_ORIGIN_s7_r5_std
Generating ARR_DELAY_ORIGIN_s14_r5_std
Generating ARR_DELAY_ORIGIN_s7_r10_std
Generating ARR_DELAY_ORIGIN_s14_r10_std
Generating ARR_DELAY_ORIGIN_s7_r1_median
Generating ARR_DELAY_ORIGIN_s14_r1_median
Generating ARR_DELAY_ORIGIN_s7_r15_median
Generating ARR_DELAY_ORIGIN_s14_r15_median
Generating ARR_DELAY_ORIGIN_s7_r30_median
Generating ARR_DELAY_ORIGIN_s14_r30_median
Generating ARR_DELAY_ORIGIN_s7_r1_std
Generating ARR_DELAY_ORIGIN_s14_r1_std
Generating ARR_DELAY_ORIGIN_s7_r15_std
Generating ARR_DELAY_ORIGIN_s14_r15_std
Generating ARR_DELAY_ORIGIN_s7_r30_std
Generating ARR_DELAY_ORIGIN_s14_r30_std


In [160]:
# Lag features by: OP_CARRIER_AIRLINE_ID
calculations = {}
calculations['calc8'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'ARR_DELAY','shifts':[s1,s2], 'windows':[1,5,10], 'funs':['mean']}
calculations['calc9'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'ARR_DELAY','shifts':[s2], 'windows':[10], 'funs':['median','std']}

df_ = apply_calc(df_, calculations)

Generating ARR_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_r1_mean
Generating ARR_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s14_r1_mean
Generating ARR_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_r5_mean
Generating ARR_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s14_r5_mean
Generating ARR_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_r10_mean
Generating ARR_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s14_r10_mean
Generating ARR_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s14_r10_median
Generating ARR_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s14_r10_std


In [161]:
# Convert times 
# [ARR_TIME, DEP_TIME]

df_['DEP_TIME'] = df_['DEP_TIME']/100
df_['ARR_TIME'] = df_['ARR_TIME']/100

df_['DEP_TIME'] = df_['DEP_TIME'].astype(int)
df_['ARR_TIME'] = df_['ARR_TIME'].astype(int)

In [162]:
df_['DEP_n_flights_per_hour'] = df_.groupby(['FL_DATE','DEP_TIME'])['ARR_TIME'].transform( lambda x: x.count())
df_['ARR_n_flights_per_hour'] = df_.groupby(['FL_DATE','ARR_TIME'])['DEP_TIME'].transform( lambda x: x.count())

df_['n_flights_per_hour'] = df_['DEP_n_flights_per_hour'] + df_['ARR_n_flights_per_hour'] 

df_['DEP_n_flights_cumsum'] = df_.groupby(['FL_DATE'])['DEP_n_flights_per_hour'].transform( lambda x: x.cumsum())
df_['ARR_n_flights_cumsum'] = df_.groupby(['FL_DATE'])['ARR_n_flights_per_hour'].transform( lambda x: x.cumsum())
df_['n_flights_cumsum'] = df_.groupby(['FL_DATE'])['n_flights_per_hour'].transform( lambda x: x.cumsum())

In [163]:
# Not Lagged
calculations = {}

calculations['calc12'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'ARR_DELAY','shifts':[s1], 'funs':['mean','std']}
calculations['calc13'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'ARR_DELAY','shifts':[s1], 'funs':['mean','std']}

df_ = apply_calc(df_, calculations)

Generating ARR_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_mean
Generating ARR_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_std
Generating ARR_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_mean
Generating ARR_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_std


In [164]:
# Type of delay
calculations = {}
calculations['calc1'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'CARRIER_DELAY','shifts':[s1], 'funs':['mean']}
calculations['calc2'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'LATE_AIRCRAFT_DELAY','shifts':[s1], 'funs':['median']}
calculations['calc3'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'NAS_DELAY','shifts':[s1], 'funs':['mean']}
calculations['calc4'] = {'gb_list':['FL_DATE'],'target':'WEATHER_DELAY','shifts':[s1], 'funs':['mean']}
calculations['calc5'] = {'gb_list':['OP_CARRIER_AIRLINE_ID','FL_DATE'],'target':'LATE_AIRCRAFT_DELAY','shifts':[s1], 'funs':['mean']}

calculations['calc6'] = {'gb_list':['ORIGIN_AIRPORT_ID'],'target':'WEATHER_DELAY','shifts':[s1], 'funs':['mean']}
calculations['calc7'] = {'gb_list':['ORIGIN_AIRPORT_ID'],'target':'NAS_DELAY','shifts':[s1], 'funs':['mean']}

df_ = apply_calc(df_, calculations)

Generating CARRIER_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_mean
Generating LATE_AIRCRAFT_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_median
Generating NAS_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_mean
Generating WEATHER_DELAY_FL_DATE_s7_mean
Generating LATE_AIRCRAFT_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_mean
Generating WEATHER_DELAY_ORIGIN_AIRPORT_ID_s7_mean
Generating NAS_DELAY_ORIGIN_AIRPORT_ID_s7_mean


In [165]:
drop_cols = ['CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'LATE_AIRCRAFT_DELAY', 'Unnamed: 29']
for col in drop_cols:
    del df_[col]

In [166]:
keep = list(df_.select_dtypes(include=np.number).columns)

for col in df_.select_dtypes(exclude=np.number).columns:
    print(col, len(df_[col].unique()))
    if len(df_[col].unique()) < 100:
        keep.append(col)
        le = LabelEncoder()
        df_[col] = le.fit_transform(df_[col])
        df_[col] = df_[col].astype('category')

FL_DATE 1338
OP_UNIQUE_CARRIER 1
OP_CARRIER 1
TAIL_NUM 873
ORIGIN 40
ORIGIN_CITY_NAME 39
ORIGIN_STATE_ABR 24
DEST 1
DEST_CITY_NAME 1
DEST_STATE_ABR 1
year_month 44
Day 274
ICAO 2


In [167]:
drop_cols = ['DEST_CITY_NAME','ORIGIN_CITY_NAME', 'Unnamed: 0']

for col in drop_cols:
    try:
        keep.remove(col)
    except:
        pass

In [168]:
df_.fillna(0, inplace=True)

In [169]:
train_df = df_.loc[(df_.FL_DATE > datetime.datetime(2018, 1, 1)) & (df_.FL_DATE < datetime.datetime(2019, 9, 1)), keep]
test_df = df_.loc[(df_.FL_DATE >= datetime.datetime(2019, 9, 1)) & (df_.FL_DATE <= datetime.datetime(2019, 9, 30)), keep]

In [170]:
X_train_ = X_train.copy(deep=True)
X_test_ = X_test.copy(deep=True)

In [171]:
train_df['DEP_DELAY_pred'] = reg.predict(X_train_)
test_df['DEP_DELAY_pred'] = reg.predict(X_test_)

In [172]:
train_df['DEP_DELAY_pred'] = train_df['DEP_DELAY_pred'].astype(int)
test_df['DEP_DELAY_pred'] = test_df['DEP_DELAY_pred'].astype(int)

In [173]:
TARGET = 'ARR_DELAY'
drop = ['DEP_DELAY','ARR_DELAY']

In [174]:
features = [ col for col in train_df.columns if col != TARGET and col not in drop ]

X_train = train_df[features]
y_train = train_df[TARGET]

X_test = test_df[features]
y_test = test_df[TARGET]

In [175]:
X_train.isnull().sum()[ X_train.isnull().sum() != 0]

Series([], dtype: int64)

In [176]:
reg = gbr(loss='squared_error', n_estimators=400)
reg.fit(X_train, y_train)

GradientBoostingRegressor(n_estimators=400)

In [177]:


rmse = mean_squared_error(y_train, reg.predict(X_train), squared=True)
rmse

814.788035380547

In [178]:
pd.DataFrame({'columns': X_train.columns,'feature_importance':reg.feature_importances_}).sort_values('feature_importance',ascending=False).head(25)

,columns,feature_importance
122,DEP_DELAY_pred,0.595710
105,ARR_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_mean,0.041433
85,ARR_DELAY_ORIGIN_s7_r30_median,0.023675
106,ARR_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_std,0.020704
109,NAS_DELAY_OP_CARRIER_AIRLINE_ID_FL_DATE_s7_mean,0.013427
61,ARR_DELAY_FL_DATE_s7_r15_median,0.010653
1,OP_CARRIER_FL_NUM,0.010097
86,ARR_DELAY_ORIGIN_s14_r30_median,0.008944
73,ARR_DELAY_ORIGIN_s7_r10_mean,0.008587
71,ARR_DELAY_ORIGIN_s7_r5_mean,0.008559


In [179]:
train_df_ = train_df.copy(deep=True)
train_df_['ARR_DELAY_pred'] = reg.predict(X_train)
train_df_[['DEP_DELAY','ARR_DELAY','DEP_DELAY_pred','ARR_DELAY_pred']].sort_values('ARR_DELAY', ascending=False).head(30)

,DEP_DELAY,ARR_DELAY,DEP_DELAY_pred,ARR_DELAY_pred
167736,500.0,485.0,10,31.396190
43693,486.0,474.0,10,26.936482
429083,470.0,466.0,2,24.627570
701391,396.0,466.0,64,283.780992
39143,479.0,459.0,12,361.600059
1019599,462.0,455.0,2,8.736196
590400,351.0,439.0,253,410.816978
298109,453.0,433.0,25,110.048761
1032656,412.0,431.0,56,268.615821
554615,433.0,425.0,8,1.410648


In [180]:
train_df_[['DEP_DELAY','ARR_DELAY','DEP_DELAY_pred','ARR_DELAY_pred']].sort_values('ARR_DELAY').head(30)

,DEP_DELAY,ARR_DELAY,DEP_DELAY_pred,ARR_DELAY_pred
561822,-4.0,-60.0,12,-1.072112
290537,-4.0,-53.0,6,-15.631712
45274,-2.0,-53.0,10,-9.841017
299480,0.0,-52.0,8,-15.882624
290402,-4.0,-52.0,9,-10.323550
39557,1.0,-52.0,11,-1.165879
43800,-1.0,-51.0,12,-24.520974
290354,-4.0,-51.0,9,-15.523923
295834,-8.0,-51.0,9,-8.778890
295761,-3.0,-50.0,8,-11.069142


In [181]:
# Big error 
train_df_['ARR_DELAY_error'] = train_df_['ARR_DELAY'] - train_df_['ARR_DELAY_pred']
train_df_[['DEP_DELAY','ARR_DELAY','DEP_DELAY_pred','ARR_DELAY_pred','ARR_DELAY_error']].sort_values('ARR_DELAY_error', ascending=False).head(30)

,DEP_DELAY,ARR_DELAY,DEP_DELAY_pred,ARR_DELAY_pred,ARR_DELAY_error
167736,500.0,485.0,10,31.396190,453.603810
43693,486.0,474.0,10,26.936482,447.063518
1019599,462.0,455.0,2,8.736196,446.263804
429083,470.0,466.0,2,24.627570,441.372430
554615,433.0,425.0,8,1.410648,423.589352
1075239,431.0,411.0,14,6.184702,404.815298
352130,408.0,402.0,6,-0.290388,402.290388
553904,406.0,388.0,2,-4.392240,392.392240
556408,392.0,377.0,6,4.376401,372.623599
495235,361.0,349.0,11,5.295564,343.704436


In [182]:
train_df_[['DEP_DELAY','ARR_DELAY','DEP_DELAY_pred','ARR_DELAY_pred']].sample(20)

,DEP_DELAY,ARR_DELAY,DEP_DELAY_pred,ARR_DELAY_pred
261372,4.0,-12.0,8,-0.502721
644939,24.0,15.0,9,14.964351
254237,-2.0,-21.0,6,0.701038
125000,17.0,-1.0,9,-3.673650
476569,-2.0,-8.0,24,23.268628
841424,4.0,6.0,9,7.584724
740346,-4.0,-16.0,12,4.897614
999507,-1.0,-18.0,16,7.849336
642460,63.0,65.0,14,13.346109
641859,3.0,-7.0,6,-4.462366


In [183]:
perc =[0.05, .10, .25, 0.5, 0.75, .90, 0.95]
train_df_['ARR_DELAY_error'].describe(percentiles = perc)

count    6.779300e+04
mean    -6.202169e-17
std      2.854470e+01
min     -1.670092e+02
5%      -2.691325e+01
10%     -2.184083e+01
25%     -1.409137e+01
50%     -5.778134e+00
75%      5.165825e+00
90%      2.464548e+01
95%      4.714879e+01
max      4.536038e+02
Name: ARR_DELAY_error, dtype: float64

In [184]:
finish

NameError: name 'finish' is not defined

In [ ]:
reg = lgbm.LGBMRegressor(objective='rmse', n_estimators=400)
reg.fit(X_train, y_train)

In [ ]:
train_df_ = train_df.copy(deep=True)
train_df_['pred'] = reg.predict(X_train)
train_df_[['ARR_DELAY','pred']].sort_values('ARR_DELAY', ascending=False).head(30)

,ARR_DELAY,pred
49775,485.0,415.040911
54432,474.0,437.624527
27961,466.0,440.571741
121878,466.0,424.033499
53728,459.0,424.567348
107426,455.0,431.884757
96953,439.0,403.761763
61193,433.0,395.227514
146185,431.0,375.421558
18727,425.0,350.585559


In [ ]:
stop

NameError: name 'stop' is not defined

In [ ]:
# With SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import GridSearchCV


In [ ]:
oversample = SMOTE()
X_train_ov, y_train_ov = oversample.fit_resample(X_train, y_train)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 1, n_neighbors = 6

In [ ]:
print('Before Sampling')
counter = Counter(y_train)
for k,v in counter.items():
    per = v / len(y_train) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

print('\n After Sampling')
counter = Counter(y_train_ov)
for k,v in counter.items():
    per = v / len(y_train) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

Before Sampling
Class=1, n=96856 (14.902%)
Class=0, n=553079 (85.098%)

 After Sampling
Class=1, n=553079 (85.098%)
Class=0, n=553079 (85.098%)


In [ ]:
rf = lgbm.LGBMClassifier(metric='roc_auc')

param_dist = {"max_depth": [30, 45, 50],
                  "n_estimators": [150, 200, 250, 300]
                 }
grid_search = GridSearchCV(rf, n_jobs=-1, param_grid=param_dist, cv = 5, scoring='roc_auc', verbose=20)
grid_search.fit(X_train_ov, y_train_ov)
smote_model = grid_search.best_estimator_

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/Users/carlosperezricardo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
LGBMClassifier(max_depth=30, metric='roc_auc', n_estimators=300)
TRAIN (oversample): 0.9319798799086567
TRAIN: 0.6536715539369677
TEST: 0.5717710544939844


In [ ]:
smote_model = lgbm.LGBMClassifier(metric='roc_auc', n_estimators=300, max_depth=50)
smote_model.fit(X_train_ov, y_train_ov)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


LGBMClassifier(max_depth=50, metric='roc_auc', n_estimators=300)

In [ ]:
print(smote_model)
y_pred_train_ = smote_model.predict_proba(X_train_ov)[:, 1]
y_pred_train = smote_model.predict_proba(X_train)[:, 1]
y_pred_test = smote_model.predict_proba(X_test)[:, 1]

print("TRAIN (oversample):", roc_auc_score(y_train_ov, y_pred_train_))
print("TRAIN:", roc_auc_score(y_train, y_pred_train))
print("TEST:", roc_auc_score(y_test, y_pred_test))

LGBMClassifier(max_depth=50, metric='roc_auc', n_estimators=300)
TRAIN (oversample): 0.9741408816810191
TRAIN: 0.8538919943665835
TEST: 0.7909392453378772


In [ ]:
confusion_matrix(y_train, smote_model.predict(X_train))

array([[545714,   7365],
       [ 65573,  31283]])